In [27]:
import matplotlib.pyplot as plt
import pandas as pd
from string import Template

from blm_activity_db import BlmActivityDb, Community
from tweet_mgr import Stance
from tweet_sentiment import PronounCounts


In [20]:
db = BlmActivityDb()
col_names = ["Stance", "NumTweets", "FirstSingularUsage", "FirstPluralUsage", "SecondUsage", "ThirdUsage"]
pronoun_columns = ", a.".join(col_names)
query = \
f"SELECT {pronoun_columns} "\
"FROM Community c JOIN AccountActivity a "\
"ON c.PeriodId = a.PeriodId and c.CommunityId = a.CommunityId "\
"WHERE c.PeriodId = ? and c.Stance in (-1, 1) "

def get_pronoun_counts(period: int):
    activist_pc, counter_pc = PronounCounts(), PronounCounts()
    activist_tweets, counter_tweets = 0, 0
    with db.conn:
        cur = db.conn.cursor()
        cur.execute(query, (period,))
        row = cur.fetchone()
        while row:
            num_tweets = row[1]
            if row[0] == Stance.Protest:
                activist_tweets += num_tweets
                activist_pc.first_singular += num_tweets * row[2]
                activist_pc.first_plural += num_tweets * row[3]
                activist_pc.second += num_tweets * row[4]
                activist_pc.third += num_tweets * row[5]
            else:
                counter_tweets += num_tweets
                counter_pc.first_singular += num_tweets * row[2]
                counter_pc.first_plural += num_tweets * row[3]
                counter_pc.second += num_tweets * row[4]
                counter_pc.third += num_tweets * row[5]
            row = cur.fetchone()
    activist_pc /= activist_tweets
    counter_pc /= counter_tweets
    return activist_pc, counter_pc


In [30]:
template = Template("""
## Pronoun Usage for Period $period

| Stance | First Person Singular | First Person Plural | Second Person | Third Person |
|:------:|:-----:|:-----:|:-----:|:-----:|
| Activist | $afs | $afp | $as | $at |
| Counter-Protest | $cfs | $cfp | $cs |$ct |

""")
report = ""
for period in [i for i in range(1, 7)]:
    a_pc, c_pc = get_pronoun_counts(period)
    subs = {
        "period": period,
        "afs": round(a_pc.first_singular, 3), 
        "afp": round(a_pc.first_plural, 3), 
        "as": round(a_pc.second, 3), 
        "at": round(a_pc.third, 3),
        "cfs": round(c_pc.first_singular, 3), 
        "cfp": round(c_pc.first_plural, 3), 
        "cs": round(c_pc.second, 3), 
        "ct": round(c_pc.third, 3),
    }
    report += template.safe_substitute(subs)

report_path = "../data/Reports/pronoun_counts.md"
with open(report_path, "w") as f:
    f.write(report)